# Model Training Script
This script extracts data from the snowflake table, transforms the features, and then traings the model on the engeneered features. Our training script leverages MLflow for model logging.

In [35]:
#!pip install snowflake-connector-python
#!pip install mlflow mlflow==1.22.0

In [43]:
import os
from datetime import datetime
import json
import argparse
from dotenv import load_dotenv
import joblib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import snowflake.connector

from sklearn.datasets import make_regression
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, make_scorer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor

# Enable pandas to display up to 500 columns
pd.set_option('display.max_columns', 500)


#### 1.1 Load Snowflake credentials for temp user

In [44]:
# Load environment variables from a .env file
load_dotenv()

try:
    TEMP_USER = os.getenv('SNOWSQL_TEMP_USER')
    TEMP_USER_PASSWORD = os.getenv('SNOWSQL_TEMP_PWD')
    
    if not TEMP_USER:
        raise ValueError("Environment variable SNOWSQL_TEMP_USER must be set")
    if not TEMP_USER_PASSWORD:
        raise ValueError("Environment variable SNOWSQL_TEMP_PWD must be set")

except ValueError as ve:
    print(f"Error: {ve}")
    
except Exception as e:
    print(f"An unexpected error occurred: {e}")

#### 1.2 Connect to Snowflake

In [38]:
# Establish connection to Snowflake
current_time = datetime.now().strftime('%Y-%m-%d %H:%M%S')

try:
    conn = snowflake.connector.connect(
        user=TEMP_USER,
        password=TEMP_USER_PASSWORD,
        account='ygeuort-alb19263',
        warehouse='COMPUTE_WH',
        database='AIRBNB',
        schema='ODS'
    )

    print(f'Connected to Snowflake successfully at {current_time}')

except Exception as e:
    print(f'Failed to connect to Snowflake on {current_time} due to error code {e}')

Connected to Snowflake successfully at 2024-07-28 20:0317


In [39]:
def get_data(sql_query, date_columns=None):
    """
    Executes a SQL query and returns the result as a pandas DataFrame.

    Args:
        sql_query (str): SQL query to execute.

    Returns:
        df_result: Resulting DataFrame from the SQL query.
    """
    try:
        cursor = conn.cursor()
        
        cursor.execute(sql_query)

        # load data into dataframe
        df_result = cursor.fetch_pandas_all()

        # Convert column names to lowercase
        df_result.columns = map(str.lower, df_result.columns)

        # Parse specified date columns
        if date_columns:
            for col in date_columns:
                df_result[col] = pd.to_datetime(df_result[col], errors='coerce')

    finally:
        if cursor is not None:
            cursor.close()

    return df_result

#### Extract data for all markets

In [40]:
sql_query = '''
select * from listings
 '''

df_raw = get_data(sql_query)
df_raw.shape

(111052, 68)

In [41]:
# Select features and target for the model
features = ['market', 'room_type', 'accommodates', 'bathrooms', 'beds', 'latitude', 'longitude', 'amenities']
categorical_features = ['market', 'room_type']  # Features with categorical data
numerical_features = ['accommodates', 'bathrooms', 'beds', 'latitude', 'longitude']  # Features with numerical data
text_features = ['amenities']  # (Optional) Feature for text data like amenities
target = 'price'  # Target variable to predict

# Filter the dataframe to include only the selected features and target column
df = df_raw[features + [target]]

# Remove rows where the price exceeds $600 per night to avoid outliers
df = df[df[target] <= 600]

# Drop rows with missing values in the target column to ensure data integrity
df = df.dropna(subset=[target])

# Separate the features (X) and the target (y)
X = df.drop(columns=[target])
y = df[target]

# Split the data into training and test sets to evaluate model performance. Note because we have a large dataset, 111,052 obsevations, we can use a smaller test size of 10% the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [42]:
# Get model run start time
start_time = datetime.now()

# Define the pipeline
def createPipeline(numerical_features, categorical_features):
    # Define the preprocessing for numerical features
    numericalTransformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])
    
    # Define the preprocessing for categorical features
    categoricalTransformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])
    
    # Combine preprocessing for numerical and categorical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numericalTransformer, numerical_features),
            ('cat', categoricalTransformer, categorical_features)
        ])
    
    # Create the pipeline
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(max_depth=30,
                                            max_features='auto',
                                            min_samples_leaf=2,
                                            min_samples_split=2,
                                            n_estimators=300,
                                            random_state=42)) #best model after hyperparameter tuning
    ])
    return pipeline

def log_to_file(log_file, params, metrics, training_details):
    # Load existing data
    try:
        with open(log_file, 'r') as f:
            logs = json.load(f)
    except (FileNotFoundError, json.JSONDecodeError):
        logs = []

    # Append new log entry
    log_entry = {
        "params": params,
        "metrics": metrics,
        "training_details": training_details
    }
    logs.append(log_entry)

    # Write updated logs back to the file
    with open(log_file, 'w') as f:
        json.dump(logs, f, indent=4)

# Parameters and file paths
log_file = 'experiment_log.json'
model_file = 'regression_pipeline.joblib'

# Create the pipeline
pipeline = createPipeline(numerical_features, categorical_features)

# Perform cross-validation with Mean Absolute Error (MAE)
maeScores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring='neg_mean_absolute_error', n_jobs=-1)
maeScores = -maeScores  # Convert negative MAE to positive
print("Cross-validation MAE scores:", maeScores)
print("Average cross-validation MAE:", maeScores.mean())

# Perform cross-validation with Root Mean Squared Error (RMSE)
rmse_scorer = make_scorer(mean_squared_error, squared=False)  # squared=False returns RMSE
rmseScores = cross_val_score(pipeline, X_train, y_train, cv=10, scoring=rmse_scorer, n_jobs=-1)
print("Cross-validation RMSE scores:", rmseScores)
print("Average cross-validation RMSE:", rmseScores.mean())

# Parameters and metrics to log
params = {
    "model_type": str(pipeline.get_params()['regressor']),
    "numerical_features": numerical_features,
    "categorical_features": categorical_features
}
metrics = {
    "cv_mae_mean": maeScores.mean(),
    "cv_rmse_mean": rmseScores.mean(),
    "cv_mae_std": maeScores.std(),
    "cv_rmse_std": rmseScores.std()
}
training_details = {
    "train_size": len(X_train),
    "test_size": len(X_test),
    "train_duration": str(datetime.now() - start_time)
}

# Train the model on the entire training set
pipeline.fit(X_train, y_train)

# Predict and evaluate the model on the test set using MAE
y_pred = pipeline.predict(X_test)
mae_test = mean_absolute_error(y_test, y_pred)
print("Test set MAE:", mae_test)

# Calculate RMSE on the test set
rmse_test = mean_squared_error(y_test, y_pred, squared=False)  # squared=False returns RMSE
print("Test set RMSE:", rmse_test)

# Log training run results to file
metrics.update({"test_mae": mae_test, "test_rmse": rmse_test})
log_to_file(log_file, params, metrics, training_details)

# Refit the final model on the entire dataset to improve generalization by leveraging all available information
pipeline.fit(X, y)

# Save the model
joblib.dump(pipeline, model_file, compress=('gzip', 9))


Cross-validation MAE scores: [46.68859149 45.83769521 46.89561522 46.04752452 45.3387151  46.27384157
 47.72848338 46.45816514 44.7925808  46.91317849]
Average cross-validation MAE: 46.29743909095708
Cross-validation RMSE scores: [71.39870686 69.99117595 71.61107009 69.75335034 67.972491   69.71784132
 71.65582086 69.93141402 69.28409406 70.97600159]
Average cross-validation RMSE: 70.22919660817408
Test set MAE: 46.041895386942365
Test set RMSE: 69.08103970267469


['regression_pipeline.joblib']